In [1]:
import pandas as pd
import re

# Preparation of the datasets

We read the csv containing the FANTOM's annotations and we generate a dataframe.

In [2]:
df = pd.read_csv("anno-fantom.csv") #dataframe with FANTOM annotations

In [3]:
df

,T000001,p1@LOXL4,84171,17171,Q96JB6,LOXL4,lysyl oxidase like 4,gene with protein product
0,T000006,p1@PYROXD2,84795,23517,Q8N2H3,PYROXD2,pyridine nucleotide-disulphide oxidoreductase ...,gene with protein product
1,T000007,p2@PYROXD2,84795,23517,Q8N2H3,PYROXD2,pyridine nucleotide-disulphide oxidoreductase ...,gene with protein product
2,T000014,p1@HPS1,3257,5163,"Q92902,Q658M9,Q8WXE5",HPS1,"HPS1, biogenesis of lysosomal organelles compl...",gene with protein product
3,T000023,p1@HPSE2,60495,18374,Q8WWQ2,HPSE2,heparanase 2 (inactive),gene with protein product
4,T000024,p3@HPSE2,60495,18374,Q8WWQ2,HPSE2,heparanase 2 (inactive),gene with protein product
...,...,...,...,...,...,...,...,...
87548,T201777,p1@PRKY,5616,9444,NaN,PRKY,"protein kinase, Y-linked, pseudogene",pseudogene
87549,T201778,p4@PRKY,5616,9444,NaN,PRKY,"protein kinase, Y-linked, pseudogene",pseudogene
87550,T201779,p3@PRKY,NaN,9444,O43930,PRKY,"protein kinase, Y-linked, pseudogene",pseudogene
87551,T201797,"p1@TSPY4,p1@TSPY8","728403,728395","37471,37287","F8VZD1,A6NGL4,P0CV99,P0CW00,P0CW01",TSPY8,"testis specific protein, Y-linked 8",gene with protein product


The file appears to be without a header. We simply rewrite the column names because we don't need the first line.

In [4]:
df.rename(columns={'T000001':'id', 'p1@LOXL4':'gene', '84171':'n1', '17171':'n2', 'Q96JB6':'n3', 'LOXL4':'n4', 'lysyl oxidase like 4':'descrizione1', 'gene with protein product':'descrizione2'}, inplace=True)
df.set_index('id', inplace=True)

In order to find snoRNAs, we filter te dataframe. Then, we create another dataframe with just the entries that correspond to the small nucleolar RNA.

In [5]:
# These two transformations are used to avoid errors in the for loops
df['descrizione2']=df['descrizione2'].apply(str)
df['descrizione1']=df['descrizione1'].apply(str)

regexp = re.compile(r'.RNA, small nucleolar.') # We only search for rows that contain this text in the last column
toKeep = []

for index, row in df.iterrows():
    if regexp.search(row.descrizione2):
        toKeep.append(row)

regexp = re.compile(r'small nucleolar RNA.') # We only search for rows that contain this text in the second last column
for index, row in df.iterrows():
    if regexp.search(row.descrizione1):
        toKeep.append(row)

df2 = pd.DataFrame(toKeep) 
df2

,gene,n1,n2,n3,n4,descrizione1,descrizione2
T000102,p1@SNORA12,677800,32600,NaN,SNORA12,"small nucleolar RNA, H/ACA box 12","RNA, small nucleolar"
T009375,p2@SNORD97,692223,32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar"
T009376,p4@SNORD97,692223,32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar"
T009377,p1@SNORD97,692223,32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar"
T009378,p3@SNORD97,692223,32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar"
...,...,...,...,...,...,...,...
T189749,p1@SNORA65,26783,10222,NaN,SNORA65,"small nucleolar RNA, H/ACA box 65","RNA, small nucleolar"
T190879,p2@SNORD36B,NaN,10164,NaN,SNORD36B,"small nucleolar RNA, C/D box 36B","RNA, small nucleolar"
T190880,p1@SNORD36A,26815,10163,NaN,SNORD36A,"small nucleolar RNA, C/D box 36A","RNA, small nucleolar"
T191363,p1@SNHG7,84973,28254,NaN,SNHG7,small nucleolar RNA host gene 7,"RNA, long non-coding"


We can now save the newly created dataset as a csv.

In [6]:
df2.to_csv('anno-fantom-sno.csv', index_label='id', encoding='utf-8')

Now we read the second dataset, which has the annotation for the expansion, and repeat the search for the required rows, using the same regexes.

In [7]:
df3 = pd.read_csv("expansion_annotation.tsv", sep="\t", encoding='utf-8')
df3.head(5)

,hgnc_id_first,X00Annotation,short_description,description,association_with_transcript,entrezgene_id,hgnc_id,uniprot_id,symbol,name,locus_type,omim_id,lncrnadb,TID,CID
0,HGNC:17171,"chr10:100027943..100027958,-",p1@LOXL4,CAGE_peak_1_at_LOXL4_5end,"48bp_to_ENST00000260702,NM_032211,uc001kpa.1_5end",entrezgene:84171,HGNC:17171,uniprot:Q96JB6,LOXL4,lysyl oxidase like 4,gene with protein product,607318,NaN,T000001,"chr10:100027943..100027958,-"
1,HGNC:23517,"chr10:100174900..100174956,-",p1@PYROXD2,CAGE_peak_1_at_PYROXD2_5end,"0bp_to_ENST00000370575,ENST00000462874_5end",entrezgene:84795,HGNC:23517,uniprot:Q8N2H3,PYROXD2,pyridine nucleotide-disulphide oxidoreductase ...,gene with protein product,NaN,NaN,T000006,"chr10:100174900..100174956,-"
2,HGNC:23517,"chr10:100174957..100174982,-",p2@PYROXD2,CAGE_peak_2_at_PYROXD2_5end,"0bp_to_NM_032709,uc001kpc.2,uc001kpd.2,uc010qp...",entrezgene:84795,HGNC:23517,uniprot:Q8N2H3,PYROXD2,pyridine nucleotide-disulphide oxidoreductase ...,gene with protein product,NaN,NaN,T000007,"chr10:100174957..100174982,-"
3,HGNC:5163,"chr10:100206642..100206717,-",p1@HPS1,CAGE_peak_1_at_HPS1_5end,"0bp_to_ENST00000325103,ENST00000338546,ENST000...",entrezgene:3257,HGNC:5163,"uniprot:Q92902,uniprot:Q658M9,uniprot:Q8WXE5",HPS1,"HPS1, biogenesis of lysosomal organelles compl...",gene with protein product,604982,NaN,T000014,"chr10:100206642..100206717,-"
4,HGNC:18374,"chr10:100995440..100995474,-",p1@HPSE2,CAGE_peak_1_at_HPSE2_5end,84bp_to_AJ299720_5end,entrezgene:60495,HGNC:18374,uniprot:Q8WWQ2,HPSE2,heparanase 2 (inactive),gene with protein product,613469,NaN,T000023,"chr10:100995440..100995474,-"


In [8]:
# Again, we apply the type transformation to avoid errors in the for loops
df3['locus_type']=df3['locus_type'].apply(str)
df3['name']=df3['name'].apply(str)

# The regexes are the same as before
regexp = re.compile(r'.RNA, small nucleolar.')
toKeep = []

for index, row in df3.iterrows():
    if regexp.search(row["locus_type"]):
        toKeep.append(row)

regexp = re.compile(r'small nucleolar RNA.')
for index, row in df3.iterrows():
    if regexp.search(row["name"]):
        toKeep.append(row)

df3 = pd.DataFrame(toKeep)
df3.head(5)

,hgnc_id_first,X00Annotation,short_description,description,association_with_transcript,entrezgene_id,hgnc_id,uniprot_id,symbol,name,locus_type,omim_id,lncrnadb,TID,CID
54,HGNC:32600,"chr10:101997055..101997059,-",p1@SNORA12,CAGE_peak_1_at_SNORA12_5end,"0bp_to_ENST00000391162,NR_002954,uc001kqu.1_5end",entrezgene:677800,HGNC:32600,NaN,SNORA12,"small nucleolar RNA, H/ACA box 12","RNA, small nucleolar",611330,NaN,T000102,"chr10:101997055..101997059,-"
3805,HGNC:32760,"chr11:10823145..10823156,-",p2@SNORD97,CAGE_peak_2_at_SNORD97_5end,"0bp_to_ENST00000459187,NR_004403,uc009yge.2_5end",entrezgene:692223,HGNC:32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar",NaN,NaN,T009375,"chr11:10823145..10823156,-"
3806,HGNC:32760,"chr11:10823282..10823289,-",p4@SNORD97,CAGE_peak_4_at_SNORD97_5end,"-127bp_to_NR_004403,uc009yge.2_5end",entrezgene:692223,HGNC:32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar",NaN,NaN,T009376,"chr11:10823282..10823289,-"
3807,HGNC:32760,"chr11:10823311..10823327,-",p1@SNORD97,CAGE_peak_1_at_SNORD97_5end,"-156bp_to_NR_004403,uc009yge.2_5end",entrezgene:692223,HGNC:32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar",NaN,NaN,T009377,"chr11:10823311..10823327,-"
3808,HGNC:32760,"chr11:10823646..10823666,-",p3@SNORD97,CAGE_peak_3_at_SNORD97_5end,"-491bp_to_NR_004403,uc009yge.2_5end",entrezgene:692223,HGNC:32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar",NaN,NaN,T009378,"chr11:10823646..10823666,-"


As before, we save this new dataset as csv.

In [9]:
#df3.set_index('hgnc_id_first', inplace=True)
df3.to_csv("expansion_annotation_sno.csv", index_label='id', encoding='utf-8')

We add a new column which contains the gene name to the previous dataframe (df2- filtered dataset of FANTOM annotation). Later on, it will be useful.

In [10]:
df2['gene_name'] = df2['gene'].apply(lambda x: re.sub(r'..@', "", x, count=0, flags=0))
df2

,gene,n1,n2,n3,n4,descrizione1,descrizione2,gene_name
T000102,p1@SNORA12,677800,32600,NaN,SNORA12,"small nucleolar RNA, H/ACA box 12","RNA, small nucleolar",SNORA12
T009375,p2@SNORD97,692223,32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar",SNORD97
T009376,p4@SNORD97,692223,32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar",SNORD97
T009377,p1@SNORD97,692223,32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar",SNORD97
T009378,p3@SNORD97,692223,32760,NaN,SNORD97,"small nucleolar RNA, C/D box 97","RNA, small nucleolar",SNORD97
...,...,...,...,...,...,...,...,...
T189749,p1@SNORA65,26783,10222,NaN,SNORA65,"small nucleolar RNA, H/ACA box 65","RNA, small nucleolar",SNORA65
T190879,p2@SNORD36B,NaN,10164,NaN,SNORD36B,"small nucleolar RNA, C/D box 36B","RNA, small nucleolar",SNORD36B
T190880,p1@SNORD36A,26815,10163,NaN,SNORD36A,"small nucleolar RNA, C/D box 36A","RNA, small nucleolar",SNORD36A
T191363,p1@SNHG7,84973,28254,NaN,SNHG7,small nucleolar RNA host gene 7,"RNA, long non-coding",SNHG7


We read the dataset for the snoDB version 2 and visualize the content. We save its content into another dataframe (df4).

In [11]:
df4 = pd.read_csv("snoDB_All_V2.0.tsv", sep="\t", encoding='utf-8') #dataset snoDB v.2
df4

,snodb_id,ensembl_id,refseq_id,hgnc_id,ncbi_id,snornabase_id,snorna_atlas_id,snopy_id,rna_central_id,rfam_id,...,snrna_targets,lncrna_targets,protein_coding_targets,snorna_targets,mirna_targets,trna_targets,ncrna_targets,pseudogene_targets,other_targets,is_expressed
0,snoDB0001,ENSG00000221083,NaN,NaN,NaN,NaN,NaN,Homo_sapiens300648,URS00006F7F32,RF00599,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,snoDB0002,ENSG00000252404,NaN,NaN,NaN,NaN,NaN,NaN,URS000066189C,RF00424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,snoDB0003,ENSG00000239149,NR_003025.1,HGNC:32653,677885.0,SR0000241,snoID_0545,NaN,URS0000056589,RF01293,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,snoDB0004,ENSG00000252969,NaN,NaN,NaN,NaN,NaN,NaN,URS00006DED88,RF00156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,snoDB0005,ENSG00000251866,NR_135613.1,HGNC:52237,107397391.0,NaN,snoID_0577,NaN,URS00006C4777;URS0000BC45E9,RF00602,...,U12.1-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,snoDB2119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2119,snoDB2120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2120,snoDB2121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RF01960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2121,snoDB2122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


Then, this dataframe is combined with df2 (dataframe to which the column 'gene_name' was added). As a result, we have more detailed knowledge about the genes.

In [12]:
df2['gene_name']=df2['gene_name'].astype(str)
df4['gene_name']=df4['gene_name'].astype(str)

df5 = pd.merge(df2, df4, on='gene_name')

Finally, we create a csv file from the combined dataset, which comprises details about the snoRNAs from FANTOM5 and snoDB v.2.

In [13]:
df5.drop(['n4'], axis=1, inplace=True)
df5.to_csv("fantom_snodb.csv", index_label='id', encoding='utf-8')